# Casting

All data loaded using LOAD CSV will be returned as strings - you need to cast the data to an appropriate data type before being written to a property.

The types of data that you can store as properties in Neo4j include:

- String
- Integer
- Float (decimal values)
- Boolean
- Date/Datetime
- Point (spatial)
- Lists of values

Cypher functions to cast data include:


| Function | Description |
|----------|-------------|
| `toBoolean()` | Converts a string to a boolean value |
| `toFloat()` | Converts a string to a float value |
| `toInteger()` | Converts a string to an integer value |
| `toString()` | Converts a value to a string |
| `date()` | Converts a string to a date value |
| `datetime()` | Converts a string to a date and time value |

You can use the `apoc.meta.nodeTypeProperties()` function to show the data types used in the graph:

```cypher
CALL apoc.meta.nodeTypeProperties()
YIELD nodeType, propertyName, propertyTypes
```

## Person node dates

You used this Cypher statement to create the Person nodes:

```cypher
LOAD CSV WITH HEADERS FROM 'https://data.neo4j.com/importing-cypher/persons.csv' AS row
MERGE (p:Person {tmdbId: toInteger(row.person_tmdbId)})
SET
p.imdbId = toInteger(row.person_imdbId),
p.bornIn = row.bornIn,
p.name = row.name,
p.bio = row.bio,
p.poster = row.poster,
p.url = row.url,
p.born = row.born,
p.died = row.died
```

Correct the Person nodes. Run this updated query to modify the born and died properties to be Date values.

```cypher
LOAD CSV WITH HEADERS FROM 'https://data.neo4j.com/importing-cypher/persons.csv' AS row
MERGE (p:Person {tmdbId: toInteger(row.person_tmdbId)})
SET
p.imdbId = toInteger(row.person_imdbId),
p.bornIn = row.bornIn,
p.name = row.name,
p.bio = row.bio,
p.poster = row.poster,
p.url = row.url,
p.born = date(row.born),
p.died = date(row.died)
```

**Using MERGE not CREATE?**
As `MERGE` was used in this Cypher statement, you can run it multiple times without creating duplicate nodes. It will update the existing nodes with the new date values. If you used `CREATE` instead, you would create new nodes each time you ran the statement.

The Date data type allows you to extract the year, month, and day from the date. For example,


```cypher
MATCH (p:Person)
RETURN p.born.year as YearOfBirth
```

# Adding Properties

Previously, you used the following Cypher statement to import the `Movie` nodes:

```cypher
LOAD CSV WITH HEADERS
FROM 'https://data.neo4j.com/importing-cypher/movies.csv'
AS row
MERGE (m:Movie {movieId: toInteger(row.movieId)})
SET
m.tmdbId = toInteger(row.movie_tmdbId),
m.imdbId = toInteger(row.movie_imdbId),
m.released = row.released,
m.title = row.title,
m.year = row.year,
m.plot = row.plot,
m.budget = row.budget
```

The year and budget properties are not cast and therefore stored as strings. Update and run the existing import to cast the year and budget properties to integers using the toInteger() function.

```cypher
LOAD CSV WITH HEADERS
FROM 'https://data.neo4j.com/importing-cypher/movies.csv'
AS row
MERGE (m:Movie {movieId: toInteger(row.movieId)})
SET
m.tmdbId = toInteger(row.movie_tmdbId),
m.imdbId = toInteger(row.movie_imdbId),
m.released = row.released,
m.title = row.title,
m.year = toInteger(row.year),
m.plot = row.plot,
m.budget = toInteger(row.budget)
```

The movies.csv file also contains the following additional properties:

| Property | Data type | Cast function |
|----------|-----------|---------------|
| imdbRating | Float | `toFloat()` |
| poster | String | |
| runtime | Integer | `toInteger()` |
| imdbVotes | Integer | `toInteger()` |
| revenue | Integer | `toInteger()` |
| url | String | |

# Lists

**What is a Multi-value property?**

A multi-value property is a property that can hold one or more values. Neo4j represents this type of data as a list (or "StringArray").

All values in a list must have the same data type. For example:

    ["Apple", "Banana, "Orange"]

    [100, 55, 4]

The movies.csv data file contains multi-value properties, including:

- countries - the countries which produced the movie

- languages - the languages spoken in the movie

Find the countries and languages data in the movies.csv file. You will see that each property contains a list of values separated by a | character. For example:

```
USA|France|Italy|Germany
English|Mandarin|Russian
```

## Split the data into a list

This updated Movie import creates a list of countries by using the split function:

```cypher
LOAD CSV WITH HEADERS
FROM 'https://data.neo4j.com/importing-cypher/movies.csv'
AS row
MERGE (m:Movie {movieId: toInteger(row.movieId)})
SET
m.tmdbId = toInteger(row.movie_tmdbId),
m.imdbId = toInteger(row.movie_imdbId),
m.released = date(row.released),
m.title = row.title,
m.year = toInteger(row.year),
m.plot = row.plot,
m.budget = toInteger(row.budget),
m.imdbRating = toFloat(row.imdbRating),
m.poster = row.poster,
m.runtime = toInteger(row.runtime),
m.imdbVotes = toInteger(row.imdbVotes),
m.revenue = toInteger(row.revenue),
m.url = row.url,
m.countries = split(row.countries, '|')
```

The statement sets the countries property as a list by splitting the data from the CSV file by the | character.

Run the query and inspect the countries property data by writing a `MATCH` query to return the data.

```cypher
MATCH (m:Movie)
WHERE "France" IN m.countries
RETURN m
```

# Add Labels to Existing Nodes

## Add Actor label

Adding labels to existing nodes can make your graph more useful and performant.

The Person nodes in the graph represent both actors and directors. To determine if a person is an actor or director, you need to query the `ACTED_IN` or `DIRECTED` relationships. Alternatively, you could add labels to the existing nodes to distinguish between actors and directors.

You can add labels to a node using SET, the syntax is:

```cypher
MATCH (n)
SET n:Label
```

The following query would add the `Actor` label to all `Person` nodes that have acted in a movie:

```cypher
MATCH (p:Person)-[:ACTED_IN]->()
SET p:Actor
```

As there are people in the database who have acted in more than one movie, you can use `WITH DISTINCT` to ensure that each person is only labeled once. Although not essential, this will improve the performance of the statement.

```cypher
MATCH (p:Person)-[:ACTED_IN]->()
WITH DISTINCT p SET p:Actor
```

You can confirm it was successful by using the Actor label to find actors in the graph:

```cypher
MATCH (a:Actor) RETURN a LIMIT 25
````

## Add Director label

```cypher
MATCH (p:Person)-[:ACTED_IN]->()
WITH DISTINCT p SET p:Actor
```